In [1]:
import re
import pickle
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

### 1. Get Current Fantasy Points

In [2]:
pfr_fantasy_link = "https://www.pro-football-reference.com/years/2022/fantasy.htm"

def getCurrentFantasyPoints(link):
    info_dict = {'born' : None,
                    'draft': None,
                    'pos' : None}

    page = requests.get(link)
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')
    player_info  = soup.find_all('p')
    
    table = soup.find_all('table', id = 'fantasy')
    df = pd.read_html(str(table), flavor = 'html5lib')[0]
    df.columns = df.columns.get_level_values(1)
    df = df[df['Player'] != 'Player'].copy()
    df['Player'] = df['Player'].str.replace('*', '', regex = False)
    df['Player'] = df['Player'].str.replace('+', '', regex = False)
    df['PPR'] = pd.to_numeric(df['PPR'])
    df['PPR'].fillna(0, inplace= True)

    df.loc[df['Player'].str.contains('Etienne'), 'Player'] = 'Travis Etienne Jr.'
    return df[df['PPR'] > 10].copy()

currPts = getCurrentFantasyPoints(pfr_fantasy_link)
currPts = currPts[['Player','Tm','FantPos','G','PPR']]
currPts

,Player,Tm,FantPos,G,PPR
0,Patrick Mahomes,KAN,QB,17,417.4
1,Josh Jacobs,LVR,RB,17,328.3
2,Christian McCaffrey,2TM,RB,17,356.4
3,Derrick Henry,TEN,RB,16,302.8
4,Justin Jefferson,MIN,WR,17,368.7
...,...,...,...,...,...
484,Tony Jones,2TM,RB,6,10.4
497,Chris Manhertz,JAX,TE,17,10.2
500,Kyle Philips,TEN,WR,4,11.8
501,James Proche,BAL,WR,15,12.2


### 2. Get Projections

##### 2a. My Projections

In [3]:
with open('../projections/player_proj_2022.p', 'rb') as handle:
    proj = pickle.load(handle)
proj = proj[['Player','Tm','FantPos','PrvPts_PPR','AverageDraftPositionPPR','Preds','Preds_adp','Prob']]

##### 2b. ESPN Projections

In [4]:
# Games played dictionary
page = requests.get("https://www.pro-football-reference.com/")
if page.status_code == 200:
    soup = BeautifulSoup(page.content, 'html.parser')

games_played = {'2TM' : 16, '3TM' : 16}
afc = soup.find('table', id = 'AFC')
nfc  = soup.find('table', id = 'NFC')

for conference in ['AFC', 'NFC']:    
    conf = soup.find('table', id = conference)
    for row in conf.find_all('tr'):
        head = row.find_all('th', {'scope' : 'row'})
        if len(head) > 0:
            tm_name = re.sub('[^a-zA-Z]+', '', row.find('th').text)
            cols = row.find_all('td')
            gp = int(cols[0].text) + int(cols[1].text) + int(cols[2].text)
            games_played[tm_name] = gp
print(games_played)

{'2TM': 16, '3TM': 16, 'BUF': 16, 'MIA': 17, 'NWE': 17, 'NYJ': 17, 'CIN': 16, 'BAL': 17, 'PIT': 17, 'CLE': 17, 'JAX': 17, 'TEN': 17, 'IND': 17, 'HOU': 17, 'KAN': 17, 'LAC': 17, 'LVR': 17, 'DEN': 17, 'PHI': 17, 'DAL': 17, 'NYG': 17, 'WAS': 17, 'MIN': 17, 'DET': 17, 'GNB': 17, 'CHI': 17, 'TAM': 17, 'CAR': 17, 'NOR': 17, 'ATL': 17, 'SFO': 17, 'SEA': 17, 'LAR': 17, 'ARI': 17}


In [5]:
espn_to_pfr_dict = {'Ari':'ARI',
                    'Atl': 'ATL',
                    'Bal': 'BAL',
                    'Buf': 'BUF',
                    'Car': 'CAR',
                    'Chi': 'CHI',
                    'Cin': 'CIN',
                    'Cle': 'CLE',
                    'Dal': 'DAL',
                    'Den': 'DEN',
                    'Det': 'DET',
                    'GB': 'GNB',
                    'Hou': 'HOU',
                    'Ind': 'IND',
                    'Jax': 'JAX',
                    'KC': 'KAN',
                    'LAC': 'LAC',
                    'LAR': 'LAR',
                    'LV': 'LVR',
                    'Mia': 'MIA',
                    'Min': 'MIN',
                    'NE': 'NWE',
                    'NO': 'NOR',
                    'NYG': 'NYG',
                    'NYJ': 'NYJ',
                    'Phi': 'PHI',
                    'Pit': 'PIT',
                    'Sea': 'SEA',
                    'SF': 'SFO',
                    'TB': 'TAM',
                    'Ten': 'TEN',
                    'Wsh': 'WAS'}


In [6]:
def getESPNprojs(link = '../projections/projections_espn.xlsx'):
    proj_espn = pd.read_excel(link, sheet_name = 'projections', usecols = "A:E")
    proj_espn['Tm'] = proj_espn['Tm'].replace(espn_to_pfr_dict)
    proj_espn.drop('TeamPosition', axis = 1, inplace = True)
    proj_espn.loc[proj_espn['Player'].str.contains('DJ Moore'), 'Player'] = 'D.J. Moore'
    proj_espn.loc[proj_espn['Player'].str.contains('DK Met'), 'Player'] = 'D.K. Metcalf'
    proj_espn.loc[proj_espn['Player'].str.contains('Allen Robinson')] = "Allen Robinson"
    proj_espn.loc[proj_espn['Player'].str.contains('Gabe Davis'), 'Player']  = "Gabriel Davis"
    proj_espn.loc[proj_espn['Player'].str.contains('Marvin Jones'), 'Player']  = "Marvin Jones"
    proj_espn.loc[proj_espn['Player'].str.contains('Melvin Gordon'), 'Player']  = "Melvin Gordon"
    proj_espn.loc[proj_espn['Player'].str.contains('Darrell Henderson'), 'Player'] = "Darrell Henderson"
    proj_espn['Points'] = pd.to_numeric(proj_espn['Points'], errors = 'coerce')
    proj_espn['Points'].fillna(0)
    return proj_espn

proj_espn = getESPNprojs()
proj_espn.loc[proj_espn['Player'].str.contains('Allen Robinson'), 'Player']

78    Allen Robinson II
Name: Player, dtype: object

In [10]:
def mergeProjections(currPts, proj, proj_espn):
    """ 1. Merge on my projections """
    newPts = currPts.merge(proj, on = ['Player', 'Tm', 'FantPos'], how = 'left', indicator = True)
    # print(newPts.loc[newPts['_merge']!='both'].sort_values('PPR',ascending = False))
    newPts.loc[newPts['_merge']!='both', 'Preds'] = 0
    
    """ 2. Merge on ESPN projections"""
    newerPts = newPts.merge(proj_espn, left_on= ['Player','Tm','FantPos'], right_on = ['Player','Tm','Position'],  how='left', indicator = 'espn')
    # print(newerPts['espn'].value_counts())
    newerPts[newerPts['espn'] == 'left_only'].sort_values('PPR', ascending = False)

    """ 3. Mark guys without projections"""
    newerPts.loc[newerPts['_merge']!= 'both', 'Preds'] = 10
    newerPts.loc[newerPts['espn']!= 'both', 'Points'] = 10
    
    """ 4. Prorate PPR metric so far"""
    newerPts['GP'] = pd.to_numeric(newerPts['Tm'].replace(games_played))
    newerPts['PPR_adj'] = newerPts['PPR'] / newerPts['GP'] * 17

    newerPts.rename(columns = {'Points' : 'Points_ESPN'}, inplace = True)

    newerPts['combined'] = (newerPts['Preds'] + newerPts['Points_ESPN']) / 2
    return newerPts
newerPts = mergeProjections(currPts, proj, proj_espn)
newerPts
newerPts.to_excel('test.xlsx')

In [11]:
newerPts['Points_ESPN'].isnull().sum()
newerPts

,Player,Tm,FantPos,G,PPR,PrvPts_PPR,AverageDraftPositionPPR,Preds,Preds_adp,Prob,_merge,Position,Points_ESPN,espn,GP,PPR_adj,combined
0,Patrick Mahomes,KAN,QB,16,402.4,361.66,38.3,315.149123,299.704937,0.783244,both,QB,352.0,both,16,427.55000,333.574562
1,Josh Jacobs,LVR,RB,16,321.3,226.00,46.8,172.443816,166.377570,0.425497,both,RB,210.5,both,16,341.38125,191.471908
2,Justin Jefferson,MIN,WR,16,360.9,330.40,4.6,244.209287,229.145857,0.864425,both,WR,287.8,both,16,383.45625,266.004643
3,Austin Ekeler,LAC,RB,16,363.7,343.80,2.7,228.368831,217.989424,0.653951,both,RB,284.7,both,16,386.43125,256.534416
4,Travis Kelce,KAN,TE,16,306.5,262.80,16.5,198.122198,200.481549,0.752541,both,TE,227.3,both,16,325.65625,212.711099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,Tony Jones,2TM,RB,6,10.4,NaN,NaN,10.000000,NaN,NaN,left_only,NaN,10.0,left_only,16,11.05000,10.000000
429,Chris Manhertz,JAX,TE,16,10.2,19.10,173.0,26.673932,25.176465,0.006110,both,TE,28.7,both,16,10.83750,27.686966
430,Kyle Philips,TEN,WR,4,11.8,0.00,173.0,24.587746,28.664191,0.013812,both,WR,74.4,both,16,12.53750,49.493873
431,James Proche,BAL,WR,14,12.2,36.20,173.0,58.293357,28.664191,0.030584,both,NaN,10.0,left_only,16,12.96250,34.146678


In [12]:
from sklearn.metrics import r2_score

print(newerPts['Preds'].corr(newerPts['PPR_adj'])**2)
print(newerPts['Points_ESPN'].corr(newerPts['PPR_adj'])**2)
print(newerPts['combined'].corr(newerPts['PPR_adj'])**2)
print(newerPts['Preds_adp'].corr(newerPts['PPR_adj'])**2)

0.5632787758259113
0.5662534704415637
0.5841150128333324
0.5351335633165542


In [16]:
newerPts.sort_values('combined', ascending = False, inplace = True)
newerPts[newerPts['FantPos'] == 'RB'].head(60)

,Player,Tm,FantPos,G,PPR,PrvPts_PPR,AverageDraftPositionPPR,Preds,Preds_adp,Prob,_merge,Position,Points_ESPN,espn,GP,PPR_adj,combined
102,Jonathan Taylor,IND,RB,11,146.4,373.10,1.3,228.297461,219.663032,0.762487,both,RB,317.0,both,16,155.550000,272.648731
3,Austin Ekeler,LAC,RB,16,363.7,343.80,2.7,228.368831,217.989424,0.653951,both,RB,284.7,both,16,386.431250,256.534416
35,Najee Harris,PIT,RB,16,209.5,300.70,7.1,243.165879,212.732590,0.772435,both,RB,265.5,both,16,222.593750,254.332940
40,Joe Mixon,CIN,RB,13,221.7,287.90,10.1,215.876593,209.153182,0.674837,both,RB,258.9,both,15,251.260000,237.388296
19,Dalvin Cook,MIN,RB,16,235.1,206.30,7.5,215.251702,212.255057,0.642816,both,RB,258.5,both,16,249.793750,236.875851
53,Leonard Fournette,TAM,RB,15,224.7,255.60,21.9,205.983710,195.148320,0.584295,both,RB,258.7,both,16,238.743750,232.341855
59,Alvin Kamara,NOR,RB,14,199.8,234.70,10.9,205.376761,208.199601,0.629599,both,RB,254.8,both,16,212.287500,230.088380
94,D'Andre Swift,DET,RB,13,175.5,208.90,12.8,202.954288,205.936728,0.611597,both,RB,253.7,both,16,186.468750,228.327144
13,Saquon Barkley,NYG,RB,16,284.0,148.60,15.5,217.242612,202.726353,0.600826,both,RB,227.8,both,16,301.750000,222.521306
301,Javonte Williams,DEN,RB,4,42.0,204.90,20.2,205.062398,197.156238,0.611257,both,RB,239.0,both,16,44.625000,222.031199
